In [ ]:
# Path setup to resolve package imports (dynamic project root)
import sys, os

# Derive project root from current notebook directory: <project>/notebook
notebook_dir = os.getcwd()
project_root = os.path.dirname(notebook_dir)

if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [ ]:
from src.agents.tools.db_tools import db_tool_manager

tools = db_tool_manager.get_tools()
print(f"✅ {len(tools)} tools ready")
print("Tool names:", [t.name for t in tools])

In [ ]:
# COMPARISON NOTEBOOK: OPENAI vs OLLAMA (NO VECTOR DB)

import time
from langchain.agents import create_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from src.config.prompt import system_prompt, OLLAMA_REACT_PROMPT
from langchain_core.messages import HumanMessage

print("⚔️  OPENAI vs OLLAMA (NO VECTOR DB)")
print("=" * 80)

# 2. AGENT FACTORIES (No tools needed!)
def create_openai_agent():
    checkpointer = MemorySaver()
    model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    
    agent = create_agent(
        model=model,
        tools=tools,
        system_prompt=system_prompt,
        checkpointer=checkpointer
    )
    return agent, model, checkpointer

def create_ollama_agent():
    checkpointer = MemorySaver()
    model = ChatOllama(model="llama3-groq-tool-use", temperature=0)  # Fast local
    
    agent = create_agent(
        model=model,
        tools=tools,
        system_prompt=OLLAMA_REACT_PROMPT,
        checkpointer=checkpointer
    )
    return agent, model, checkpointer




In [ ]:
# 3. STREAMING TEST (Same as yours, no vector changes needed)
def stream_agent_comparison(agent, model_name, question, thread_id):
    """Enhanced streaming test with metrics (your original logic)"""
    config = {"configurable": {"thread_id": thread_id}}
    
    print(f"\n{'='*70}")
    print(f"🤖 {model_name} → '{question}'")
    print(f"{'='*70}")
    
    start_time = time.time()
    tool_steps = 0  # Will be 0 (no tools)
    full_content = ""
    
    for chunk in agent.stream(
        {"messages": [HumanMessage(content=question)]},
        config,
        stream_mode="values"
    ):
        if "messages" in chunk and chunk["messages"]:
            msg = chunk["messages"][-1]
            
            # TOOL CALLS 
            if hasattr(msg, 'tool_calls') and msg.tool_calls:
                tool_call = msg.tool_calls[0]
                print(f"🟡 [{time.time()-start_time:.1f}s] TOOL: {tool_call['name']}({tool_call['args']})")
                tool_steps += 1
            
            # TOOL RESULTS
            elif hasattr(msg, 'type') and msg.type == "tool":
                print(f"🟢 [{time.time()-start_time:.1f}s] RESULT: {msg.content[:80]}...")
            
            # STREAMING CONTENT (main output)
            elif msg.content:
                full_content += msg.content
                print(msg.content, end="", flush=True)
    
    total_time = time.time() - start_time
    print(f"\n\n📊 METRICS: {total_time:.1f}s | Reasoning Steps: {len(full_content.split('.'))} | Chars: {len(full_content)}")
    
    # Success = contains SQL keywords
    has_sql = any(keyword in full_content.upper() for keyword in ["SELECT", "FROM", "WHERE", "JOIN", "GROUP BY"])
    
    return {
        "time": total_time,
        "reasoning_steps": len(full_content.split('.')),
        "content_length": len(full_content),
        "has_sql": has_sql,
        "content": full_content.strip(),
        "success": has_sql and len(full_content) > 50
    }

# 4. RUN COMPARISON
print("\n🏁 HEAD-TO-HEAD (Pure LLM Reasoning)")
print("-" * 60)

# Create agents
openai_agent, _, openai_checkpointer = create_openai_agent()
ollama_agent, _, ollama_checkpointer = create_ollama_agent()

test_queries = [
    "Show top customers by sales"
]

comparison_results = {}

for i, query in enumerate(test_queries):
    print(f"\n📍 Query {i+1}/{len(test_queries)}: {query}")
    
    # Fresh threads
    openai_config = {"configurable": {"thread_id": f"no_vec_openai_{i}"}}
    ollama_config = {"configurable": {"thread_id": f"no_vec_ollama_{i}"}}
    
    # Test both
    openai_result = stream_agent_comparison(openai_agent, "🔵 OPENAI", query, f"openai_{i}")
    ollama_result = stream_agent_comparison(ollama_agent, "🟢 OLLAMA", query, f"ollama_{i}")
    
    comparison_results[query] = {
        "openai": openai_result,
        "ollama": ollama_result
    }

# 5. RESULTS TABLE
print("\n" + "="*120)
print("📊 NO-VECTOR RESULTS (Pure LLM Reasoning)")
print("="*120)

print(f"{'Query':<35} {'OpenAI':<10} {'Ollama':<10} {'OpenAI SQL':<10} {'Ollama SQL':<10} {'Faster':<12} {'Better SQL'}")
print("-"*120)

openai_wins, ollama_wins, ties = 0, 0, 0

for query, results in comparison_results.items():
    openai = results["openai"]
    ollama = results["ollama"]
    
    o_time, ol_time = openai["time"], ollama["time"]
    o_sql, ol_sql = openai["has_sql"], ollama["has_sql"]
    
    # Speed winner
    speed_winner = "🔵" if o_time < ol_time else "🟢" if ol_time < o_time else "🤝"
    
    # Quality winner (SQL + length)
    o_quality = (1 if o_sql else 0) + (openai["content_length"] / 1000)
    ol_quality = (1 if ol_sql else 0) + (ollama["content_length"] / 1000)
    
    if o_quality > ol_quality:
        quality_winner = "🔵"
        openai_wins += 1
    elif ol_quality > o_quality:
        quality_winner = "🟢" 
        ollama_wins += 1
    else:
        quality_winner = "🤝"
        ties += 1
    
    print(f"{query[:34]:<35} {o_time:<9.1f}s {ol_time:<9.1f}s "
          f"{o_sql:<9} {ol_sql:<9} {speed_winner:<11} {quality_winner}")

# 6. SUMMARY
print("="*60)
total_queries = len(test_queries)
print(f"OpenAI SQL success:  {sum(r['openai']['has_sql'] for r in comparison_results.values())}/{total_queries}")
print(f"Ollama SQL success: {sum(r['ollama']['has_sql'] for r in comparison_results.values())}/{total_queries}")

openai_avg_time = sum(r["openai"]["time"] for r in comparison_results.values()) / total_queries
ollama_avg_time = sum(r["ollama"]["time"] for r in comparison_results.values()) / total_queries


print(f"\n⏱️  Average Response Time:")
print(f"   OpenAI: {openai_avg_time:.1f}s")
print(f"   Ollama: {ollama_avg_time:.1f}s")

overall_quality_winner = "🔵 OPENAI" if openai_wins > ollama_wins else "🟢 OLLAMA"
print(f"\n🎉 QUALITY WINNER: {overall_quality_winner}")
